# Exercise 6, answers

## Problem 1

In [ ]:
#Let's use penalty function method, optimal solution is x^*=(1,0,0)

def f_constrained(x):
    return log(x[0]**2+1)+x[1]**4+x[0]*x[2],[x[0]**3-x[1]**2-1,x[0],x[2]],[]

In [ ]:
def alpha(x,f):
    (_,ieq,eq) = f(x)
    return sum([min([0,ieq_j])**2 for ieq_j in ieq])+sum([eq_k**2 for eq_k in eq])

In [ ]:
def penalized_function(x,f,r):
    return f(x)[0] + r*alpha(x,f)

In [ ]:
import numpy as np
from scipy.optimize import minimize
r = 1
x_start = [-10,-3,-5]
x_old = np.array([float('inf')]*3)
x_new = x_start
while np.linalg.norm(x_new-x_old)>0.000001:
    res = minimize(lambda x:penalized_function(x,f_constrained,r),
               x_start,method='Nelder-Mead')
    x_old = x_new
    x_new = np.array(res.x)
    r = 2*r #r+1
print(x_new, r)
print(f_constrained(x_new))

In [ ]:
# Another example using Pyomo

from pyomo.environ import *

model = ConcreteModel()
#Three variables
model.x = Var([1,2,3])
#Objective function including powers and logarithm
model.OBJ = Objective(expr = log(model.x[1]**2+1)+model.x[2]**4
                      +model.x[1]*model.x[3]) #Objective function
model.constr = Constraint(expr = model.x[1]**3-model.x[2]**2>=1)
model.box1 = Constraint(expr = model.x[1]>=0)
model.box2 = Constraint(expr = model.x[3]>=0)

from pyomo.opt import SolverFactory #Import interfaces to solvers

opt = SolverFactory("ipopt") #Use ipopt

res = opt.solve(model, tee=True) #Solve the  problem and print the output

print("Optimal solutions is ")
model.x.display()
print("Objective value at the optimal solution is ")
model.OBJ.display()

## Problem 2

The set of Pareto optimal solutions is $\{(t,1-t):t\in[0,1]\}$.

Let us denote set of Pareto optimal solutions by $PS$ and show that $PS=\{(t,1-t):t\in[0,1]\}$.

$PS\supset\{(t,1-t):t\in[0,1]\}$:

Let's assume that there exists $t\in[0,1]$, which is not Pareto optimal. Then there exists $x=(x_1,x_2)\in\mathbb R^2$ and $t\in[0,1]$ such that
$$
\left\{
\begin{align}
\|x-(1,0)\|^2<\|(t,1-t)-(1,0) \|^2,\text{ and}\\
\|x-(0,1)\|^2\leq\|(t,1-t)-(0,1) \|^2
\end{align}
\right.
$$
or
$$
\left\{
\begin{align}
\|x-(1,0)\|^2\leq\|(t,1-t)-(1,0) \|^2,\text{ and}\\
\|x-(0,1)\|^2<\|(t,1-t)-(0,1)\|^2.
\end{align}
\right.
$$

But in both cases

$$
\sqrt{2} = \|(1,0)-(0,1)\|\\
\leq \|(1,0)-x\|+\|x-(0,1)\|\\
< \|(t,1-t)-(1,0) \|+\|(t,1-t)-(0,1) \|\\
= \|(1,0)-(0,1)\| =\sqrt{2}.
$$
because the point $(t,1-t)$ is on the straight line from $(1,0)$ to $(0,1)$.

Thus, neither one of the requirements of non-Pareto optimality can hold. Thus, the point is Pareto optimal.

$PS\subset\{(t,1-t):t\in[0,1]\}$:

Let's assume a Pareto optimal solution $x$. This follows from the triangle inequality.

## Problem 3

Ideal:

To solve
$$
\min \|x-(1,0)\|^2\\
\text{s.t. }x\in \mathbb R^2.
$$
The solution of this problem is naturally $x = (1,0)$ and the minimum is $0$. Minimizing the second objective give $x=(0,1)$ and the minimum is again $0$. Thus, the ideal is $(0,0)$.

Now, the problem has just two objectives and thus, we get the components of the nadir by optimizing
$$
\min f_1(x)\\
\text{s.t. }f_2(x)\leq z^{ideal}_2
$$
and
$$
\min f_2(x)\\
\text{s.t. }f_1(x)\leq z^{ideal}_1.
$$

The solution of this problem is Pareto optimal because of the epsilon constraint method and also because the other one of the objectives is at the minimum and the other one cannot be grown with growing the other. Thus, the components of the nadir are at least the optimal values of the above optimization problems.

On the other hand, the components of the nadir have to be at most the optimal values of the above optimization problems, because if this was not the case, then the solution would not be Pareto optimal.

By solving these optimization problems, we get nadir (2,2).

## Problem 4

In [ ]:
def prob(x):
    return [(x[0]-1)**2+x[1]**2,x[0]**2+(x[1]-1)**2]

In [ ]:
import numpy as np
from scipy.optimize import minimize
def weighting_method(f,w):
    points = []
    for wi in w:
        res=minimize(
            #weighted sum
            lambda x: sum(np.array(wi)*np.array(f(x))), 
            [0,0],
            options = {'disp':False})
        points.append(res.x)
    return points

In [ ]:
w = np.random.random((500,2)) #500 random weights
wn = w
for i in range(len(w)):
    s = sum(w[0])
    for j in range(2):
        wn[i][j] = w[i][j]/s
#help(minimize)
repr_ws = weighting_method(prob,wn)

Another alternative by using Pyomo:

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory #Import interfaces to solvers

def weighting_method_pyomo(f,w):
    points = []
    for wi in w:
        model = ConcreteModel()
        model.x = Var([0,1])
        #weighted sum
        model.obj = Objective(expr = wi[0]*f(model.x)[0]+wi[1]*f(model.x)[1])
        opt = SolverFactory("ipopt") #Use ipopt
        #Combination of expression and function
        res=opt.solve(model) #Solve the problem
        points.append([model.x[0].value,model.x[1].value]) #We should check for optimality...
    return points

In [ ]:
import numpy as np
w = np.random.random((500,2)) #500 random weights
repr = weighting_method_pyomo(prob,w)

**Plot the solutions in the objective space**

In [ ]:
import matplotlib.pyplot as plt
def visualize_representation(func,repr):
    f_repr_ws = [prob(repri) for repri in repr]
    fig = plt.figure()
    plt.scatter([z[0] for z in f_repr_ws],[z[1] for z in f_repr_ws])
    return plt

In [ ]:
visualize_representation(prob,repr_ws)

**Plot the solutions in the decision space**

In [ ]:
import matplotlib.pyplot as plt
def visualize_decisions(repr):
    fig = plt.figure()
    plt.scatter([x[0] for x in repr],[x[1] for x in repr])
    return plt

In [ ]:
visualize_decisions(repr_ws).show()

**What do we notice?**

In this problem, the weighting method works. This is because the objective functions are convex.

Working here means that the method produces an even representation of the whole Pareto optimal set.